# Aplica el algoritmo k-medias con este notebook en Python
En este *notebook* aprenderás a ejecutar el algoritmo k-medias (*k-means*), disponible en las librerías *PyClustering* y *scikit-learn*. Veremos cómo generar un conjunto de datos y aplicarle este algoritmo para obtener distintas particiones.
## 1. Ejecutar el algoritmo k-medias en ***PyClustering***
En la primera parte del *notebook* utilizaremos la librería *PyClustering*. Es importante familiarse con su [documentación en línea](https://pyclustering.github.io/docs/0.10.1/html/index.html) para conocer los métodos disponibles y sus estructuras de datos. En primer lugar, vamos a importar los paquetes que necesitaremos:

In [ ]:
# La primera vez que se vaya a ejecutar este notebook es necesario instalar la librería pyclustering
!pip install pyclustering
from pyclustering.cluster.kmeans import kmeans, kmeans_visualizer
from pyclustering.cluster.center_initializer import random_center_initializer
import numpy as np
import matplotlib.pyplot as plt

Vamos a generar un conjunto de datos con 50 instancias de dos variables y cuyos valores son aleatorios. Para ello, hacemos uso de la función *random* de *numpy*. Representamos este conjunto con *matplotlib* para ver cómo se han distribuido los datos.

In [ ]:
tam = 50
datos = np.random.random((tam,2))
x = datos[0:tam,0]
y = datos[0:tam,1]
plt.scatter(x, y)
plt.show()

Una vez tenemos los datos, tenemos que obtener los centroides iniciales por medio de un objeto *initializer*. Vamos a utilizar la inicialización aleatoria, a la cual le tenemos que indicar el número de grupos (k=3). A continuación, ya podemos crear una instancia del algoritmo k-medias utilizando como parámetros el conjunto de datos y la inicialización de centroides. De momento, dejamos el resto de parámetros por defecto.

In [ ]:
k = 3
centroides_iniciales = random_center_initializer(datos, k).initialize()
print(centroides_iniciales)
alg_kmedias = kmeans(datos, centroides_iniciales)

Para ejecutar el análisis de grupos, invocamos a la función *process*.

In [ ]:
alg_kmedias.process()

Tras ejecutar el algoritmo, podemos obtener los grupos y los centroides. Podemos comprobar que los centroides finales no son los mismos que los iniciales (aleatorios).

In [ ]:
# Devuelve un array de k elementos, donde cada elemento continene el índice de las instancias asignadas al grupo k
grupos = alg_kmedias.get_clusters()
print(grupos)
# Deveuelve las coordenadas de los centroides en un array de k elementos
centroides_finales = alg_kmedias.get_centers()
print(centroides_finales)

*PyClustering* nos proporciona un visualizador específico para el algoritmo k-medias con el que representar gráficamente la partición de grupos encontrada y los centroides.

In [ ]:
grafico = kmeans_visualizer.show_clusters(datos, grupos, centroides_finales)

## 2. Parametrizar el algoritmo k-medias en ***PyClustering***
Tras ver cómo se ejecuta el algoritmo k-medias con sus parámetros por defecto, vamos a aprender cómo cambiar algunos de ellos para adaptar su comportamiento a otro tipo de distancias y datos.

*PyClustering* nos proporciona el paquete [utils.metric](https://pyclustering.github.io/docs/0.10.1/html/dd/dbc/namespacepyclustering_1_1utils_1_1metric.html) con otras definiciones de distancias. Si en lugar de utilizar la distancia Euclídea (por defecto), queremos utilizar la distancia Manhattan, tenemos que declararla y pasarla como parámetro al inicializar k-medias.

In [ ]:
from pyclustering.utils import distance_metric, type_metric
dist_manhattan = distance_metric(type_metric.MANHATTAN)
alg_kmedias = kmeans(datos, centroides_iniciales, metric=dist_manhattan)
alg_kmedias.process()


Como hemos partido de la misma inicialización de centroides, podemos comprobar cómo la asignación de grupos se ve afectada por el cambio de distancia.

In [ ]:
grupos = alg_kmedias.get_clusters()
centroides_finales = alg_kmedias.get_centers()
grafico = kmeans_visualizer.show_clusters(datos, grupos, centroides_finales)

*PyClustering* nos permite incluso utilizar nuestra propia función de distancia para realizar el análisis de grupos. A continuación, vamos a modificar nuestros datos para que tomen variables discretas y definiremos nuestra propia función de distancia.

In [ ]:
# Array de valores discretos entre 0 y 10
max_valor = 10
datos = np.random.randint(max_valor, size=(tam, 2))
x = datos[0:tam,0]
y = datos[0:tam,1]
plt.scatter(x, y)
plt.show()

Nuestra función de distancia va a ponderar la proximidad en la coordenada x respecto a la coordenada y.

In [ ]:
def dist_xy(punto1, punto2):
    tam = len(punto1)
    dist = 0
    for i in range(0, tam):
        dist += 10*np.abs((punto1[0] - punto2[0])) + np.abs((punto1[1]-punto2[1])) 
    return dist

Ahora podemos indicarle a k-medias que utilice nuestra función para calcular las distancias durante el análisis de grupos. Además, vamos a asignar un número de grupos más grande. 

In [ ]:
dist_kmedias = distance_metric(type_metric.USER_DEFINED, func=dist_xy)
k=10
centroides_iniciales = random_center_initializer(datos, k).initialize()
alg_kmedias = kmeans(datos, centroides_iniciales, metric=dist_kmedias)
alg_kmedias.process()

Tras ejecutar el algoritmo, recuperamos los grupos y los centroides para visualizaros. Vemos que la agrupación tiene en cuenta la coincidencia o proximidad en los valores de la coordenada x para agrupar las instancias, aunque la diferencia en los valores de la coordenada y sean grandes.

In [ ]:
grupos = alg_kmedias.get_clusters()
centroides_finales = alg_kmedias.get_centers()
grafico = kmeans_visualizer.show_clusters(datos, grupos, centroides_finales)

## 3. Evaluar el algoritmo k-medias en ***PyClustering***
*PyClustering* nos proporciona una implementación del [método "del codo"](https://pyclustering.github.io/docs/0.10.1/html/d3/d70/classpyclustering_1_1cluster_1_1elbow_1_1elbow.html#details) para evaluar el rendimiento del algoritmo k-medias con varios valores de k. Lo primero que debemos hacer es importar la clase que lo implementa y definir el rango de valores de k a evaluar.

In [ ]:
from pyclustering.cluster.elbow import elbow
k_min, k_max = 1, 10

Vamos a crear un nuevo conjunto de datos aleatorio con variables continuas.

In [ ]:
tam = 50
datos = np.random.random((tam,2))
x = datos[0:tam,0]
y = datos[0:tam,1]
plt.scatter(x, y)
plt.show()

Ahora podemos crear una instancia del método "del codo", la cual requiere especificar la muestra de datos y el rango de k como parámetros. Además, debemos indicarle que utilice el inicializador aleatorio de centroides (por defecto utiliza otro que estudiaremos la próxima semana). Una vez configurado, lo ejecutamos invocando a la función *process*, que internamente ejecutará *k-means* tantas veces como valores de k se hayan indicado.

In [ ]:
alg_codo = elbow(datos, k_min, k_max, initializer=random_center_initializer)
alg_codo.process()

Una vez completado el análisis, podemos obtener los valores de SSE (llamado wce, *within-cluster errors*) para cada valor de k, así como el valor de k recomendado.

In [ ]:
error_k = alg_codo.get_wce()
print(error_k)
mejor_k = alg_codo.get_amount()
print(mejor_k)


Para representar los valores, utilizamos *matplotlib*. Creamos el gráfico por medio de una función para poder reutilizarlo más adelante.

In [ ]:
def visualizar_metodo_codo(rango_k, error_k):
  plt.plot(rango_k, error_k, '-o')
  plt.show()

Invocamos a la función con el rango de valores adecuado y el array de errores obtenido con *PyClustering*.

In [ ]:
rango_k = np.arange(k_min, k_max+1)
visualizar_metodo_codo(rango_k, error_k)


## 4. Ejecutar el algoritmo k-medias en ***scikit-learn***
Partiendo del último conjunto de datos aleatorio creado, vamos a cambiar probar la implementación del [algoritmo k-medias](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) de *scikit-learn*. Primero incluimos los paquetes necesarios.

In [ ]:
from sklearn.cluster import KMeans

A continuación, creamos la instancia del algoritmo, indicando el valor de k como parámetro. Elegimos el mejor valor de k según el método "del codo", aunque no necesariamente obtendremos la misma partición ya que la inicialización aleatoria será diferente. En *scikit-learn* podemos fijar el valor de la semilla aleatoria (parámetro *random_state*) para que si realizamos una nueva ejecución, el resultado sea el mismo. Además, vamos a indicar que realice una inicialización aleatoria de los centroides.

In [ ]:
k = mejor_k
alg_kmedias_sklearn = KMeans(n_clusters=k, random_state=0, init='random')

Ya podemos ejecutar el algoritmo para "ajustar" la partición de grupos, utilizando para ello la función *fit*.

In [ ]:
alg_kmedias_sklearn.fit(datos)

Los resultados que podemos extraer de la partición son dos: el número de grupo asignado a cada punto (llamado etiqueta) y la posición de los centroides.

In [ ]:
etiquetas = alg_kmedias_sklearn.labels_
print(etiquetas)
centroides = alg_kmedias_sklearn.cluster_centers_
print(centroides)

Si queremos representar la partición, tenemos que crear manualmente el gráfico con *matplotlib*

In [ ]:
# Esto dibujará los puntos con círculos coloreados según la partición asignada
plt.scatter(x, y, c=etiquetas)
# Esto dibujará los centroides con un icono de estrella, y colorado según la partición
colores = np.arange(0, k)
plt.scatter(centroides[0:k,0], centroides[0:k,1], marker="*", c=colores)
plt.show()

## 5. Evaluar el algoritmo k-medias en ***scikit-learn***
En *scikit-learn* no disponemos de una implementación del método "del codo", pero podemos realizarla nosotros mismos invocando al algoritmo k-medias con varios valores de k. Para cada valor, podemos extraer el valor de SSE (llamado *inertia*).

In [ ]:
error_k = np.zeros(k_max+1-k_min)
for k in range(k_min, k_max+1):
  alg_kmedias_sklearn = KMeans(n_clusters=k, random_state=0)
  alg_kmedias_sklearn.fit(datos)
  error_k[k-k_min] = alg_kmedias_sklearn.inertia_

Para visualizar los valores, invocamos a nuestra función que nos representa los valores de error frente a los valores de k. 

In [ ]:
visualizar_metodo_codo(rango_k, error_k)